In [ ]:
# pip install easyocr
from warnings import filterwarnings
filterwarnings("ignore")
import easyocr
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext('train_images/619cPZMqL7L.jpg',detail=0)
result_string = ' '.join(result)
print(result_string)